In [1]:
%load_ext autoreload
%autoreload 2

import pandas as pd
pd.set_option('display.max_columns', None)
pd.set_option('display.max_colwidth', -1)
import ast
import numpy as np
import matplotlib.pyplot as plt

from sklearn.preprocessing import StandardScaler
from sklearn.decomposition import PCA

import seaborn as sns

import dspace
import dspace.plotutils

from utils import int_model, draw_single_pheno_samples, check_oscillation, \
                  process_check_plot_pset, count_parameter_occurence, merge_two_dicts, \
                  random_sample_phenotype, update_stability, load_model_variables, build_analyse_design_space

SMALL_SIZE = 18
MEDIUM_SIZE = 22
BIGGER_SIZE = 26
plt.rc('font', size=SMALL_SIZE)          # controls default text sizes
plt.rc('axes', titlesize=MEDIUM_SIZE)     # fontsize of the axes title
plt.rc('axes', labelsize=MEDIUM_SIZE)    # fontsize of the x and y labels
plt.rc('xtick', labelsize=MEDIUM_SIZE)    # fontsize of the tick labels
plt.rc('ytick', labelsize=MEDIUM_SIZE)    # fontsize of the tick labels
plt.rc('legend', fontsize=SMALL_SIZE)    # legend fontsize
plt.rc('figure', titlesize=MEDIUM_SIZE)  # fontsize of the figure title

In [2]:
model_list = ['Design 3','Design 4','Design 5','Design 6','Design 7','Design 8','Design 9']

pnames = {}

all_pvals = {} # will contain pvals_model (below) for each model

path = '../LCs/summary_parameter_values_and_phenotype_dominance.xlsx'
with pd.ExcelWriter(path, engine='xlsxwriter') as writer:
    for j, model in enumerate(model_list):
        # load design space model
        scipy_model, pset, variables, y0, f, constraints, parbounds, latex_symbols, varnames = load_model_variables(model.replace('D','d').replace(' ','_'))
        ds, valid_cases = build_analyse_design_space(model, f, constraints, latex_symbols)

        path = '../LCs/'+model+'/LC_parameters'
        df_LC = pd.read_pickle(path)

        # require correct peak sequence
        df_LC = df_LC[df_LC['Peak sequence'].isin(['s, x, y, z','x, y, z, s', 'y, z, s, x', 'z, s, x, y'])]

        df_LC = df_LC.apply(lambda x: dict(x['Parameters']),axis=1)

        df_pars = pd.DataFrame()
        df_dominance = pd.DataFrame()

        for i, row in enumerate(df_LC):
            # combine the parameters 
            new_df = pd.DataFrame([df_LC.iloc[i]], index=[df_LC.index[i]])
            df_pars = df_pars.append(new_df)
            
            # find the dominant parameters in the current row's phenotype
            phenotype = df_LC.index[i]
            
            df_dominance_pheno = count_parameter_occurence(ds, [phenotype])
            df_dominance_pheno.index = [df_LC.index[i]]
            if phenotype not in df_dominance.index.tolist():
                df_dominance = df_dominance.append(df_dominance_pheno)

                
        # drop "1" 
        df_dominance = df_dominance.drop(labels='1', axis='columns')
        
        # Export to Excel
        df_pars.to_excel(writer, sheet_name=model+"_parameter_values")
        df_dominance.to_excel(writer, sheet_name=model+"_dominance")

    writer.save()              